In [ ]:
from Train import train
from DynamicsModel import model as DynamicsModel
from env1.RealisticPendulum import RealisticPendulum as env1
import jax 
import jax.numpy as jnp
from sklearn.model_selection import train_test_split
import orbax.checkpoint as ocp
import os 


In [ ]:
# Load Data:
with jnp.load('Data/data.npz') as data:
    X_data1 = data['observation_t']
    X_data2 = data['action_t']
    X_data = jnp.concatenate((X_data1, X_data2), axis=1)
    y_data = data['observation_tp']
    print(X_data.shape, y_data.shape)
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=0)
print(X_data[:10,:], y_data[:10,:])
rng = jax.random.PRNGKey(0)

In [ ]:
params = train(rng, X_train, y_train, X_test, y_test, observation_size = 4,action_size = 1, learning_rate=0.0001, num_epochs=100, batch_size=64)

In [ ]:
from env1.RealisticPendulum import RealisticPendulum as env1

env1 = env1()
key = jax.random.key(0)

State = env1.reset(key)
State1 = env1.step(State, 1.0)
#print(State.pipeline_state)
#State.pipeline_state.q = jnp.array([0.0,0.0],dtype=jnp.float32)
qqd = jnp.concatenate((jnp.expand_dims(State.pipeline_state.q,axis=1),jnp.expand_dims(State.pipeline_state.qd, axis=1)), axis=0)[:,0]
qqd10 = jnp.concatenate((jnp.expand_dims(State1.pipeline_state.q,axis=1),jnp.expand_dims(State1.pipeline_state.qd, axis=1)), axis=0)

model = DynamicsModel(4,5)
input = jnp.concatenate((qqd, jnp.array([1.0])), axis=0)
qqd11 = model.apply({"params":params}, input )
print(qqd10, qqd11)



In [ ]:
ckpt_dir = os.path.abspath('./params')
checkpointer = ocp.AsyncCheckpointer(ocp.StandardCheckpointHandler())
path = ocp.test_utils.erase_and_create_empty(ckpt_dir)
checkpointer.save(path / str(1), args=ocp.args.StandardSave(params))
